# Sales Data Exploration

This notebook is for exploring the sales dataset. We will perform initial data loading, cleaning, and visualization to understand the data's structure, identify patterns, and derive preliminary insights.

## 1. Setup and Load Libraries

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Configure visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.options.display.max_columns = None # Show all columns
pd.options.plotting.backend = 'plotly' # Use Plotly for pandas plotting

## 2. Load Data

Load the sample sales data. Adjust the `file_path` if your data is located elsewhere. We're assuming the notebook is in a `notebooks/` directory and the data is in a parallel `data/` directory.

In [ ]:
file_path = '../data/sample_sales_data.csv' # Adjust path as needed
try:
    df_sales = pd.read_csv(file_path)
    print("Data loaded successfully!")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the path.")
    # Create a dummy DataFrame for demonstration if file is not found
    data = {
        'OrderID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'OrderDate': pd.to_datetime(['2024-01-15', '2024-01-17', '2024-01-20', '2024-02-10', '2024-02-12', '2024-03-05', '2024-03-10', '2024-04-01', '2024-04-05', '2024-04-10']),
        'ProductCategory': ['Electronics', 'Books', 'Electronics', 'Clothing', 'Books', 'Electronics', 'Clothing', 'Home Goods', 'Books', 'Electronics'],
        'ProductName': ['Laptop', 'Python Programming', 'Smartphone', 'T-Shirt', 'Data Science Handbook', 'Tablet', 'Jeans', 'Desk Lamp', 'Statistics', 'Monitor'],
        'Quantity': [2, 10, 5, 20, 5, 3, 15, 8, 12, 2],
        'UnitPrice': [1200, 45, 800, 25, 55, 600, 70, 30, 50, 300],
        'Salesperson': ['Alice', 'Bob', 'Alice', 'Carol', 'Bob', 'Alice', 'David', 'Carol', 'Bob', 'Alice'],
        'Region': ['North', 'West', 'North', 'East', 'West', 'North', 'South', 'East', 'West', 'North']
    }
    df_sales = pd.DataFrame(data)
    df_sales['TotalSale'] = df_sales['Quantity'] * df_sales['UnitPrice']
    print("Loaded dummy data for demonstration.")

## 3. Initial Data Inspection

In [ ]:
print("First 5 rows:")
display(df_sales.head())

In [ ]:
print("\nLast 5 rows:")
display(df_sales.tail())

In [ ]:
print("\nDataframe Info:")
df_sales.info()

In [ ]:
print("\nDescriptive Statistics:")
display(df_sales.describe(include='all')) # include='all' for both numeric and object types

In [ ]:
print(f"\nShape of the dataframe: {df_sales.shape}")

In [ ]:
print("\nMissing values per column:")
display(df_sales.isnull().sum())

In [ ]:
print(f"\nNumber of duplicated rows: {df_sales.duplicated().sum()}")

## 4. Data Cleaning and Preprocessing

Based on the initial inspection, perform necessary cleaning steps.

In [ ]:
# Convert 'OrderDate' to datetime if not already (should be handled by read_csv's parse_dates or done manually)
if 'OrderDate' in df_sales.columns and not pd.api.types.is_datetime64_any_dtype(df_sales['OrderDate']):
    df_sales['OrderDate'] = pd.to_datetime(df_sales['OrderDate'])
    print("'OrderDate' converted to datetime.")

# Example: Handle missing values (if any were found)
# For numerical columns, you might fill with mean or median
# For categorical columns, you might fill with mode or a specific placeholder
# e.g., df_sales['SomeNumericColumn'].fillna(df_sales['SomeNumericColumn'].median(), inplace=True)
# e.g., df_sales['SomeCategoricalColumn'].fillna('Unknown', inplace=True)

# Example: Remove duplicates (if any significant number were found and are actual duplicates)
# df_sales.drop_duplicates(inplace=True)

# Feature Engineering: Extract useful features from OrderDate
if 'OrderDate' in df_sales.columns:
    df_sales['Year'] = df_sales['OrderDate'].dt.year
    df_sales['Month'] = df_sales['OrderDate'].dt.month
    df_sales['MonthName'] = df_sales['OrderDate'].dt.month_name()
    df_sales['Day'] = df_sales['OrderDate'].dt.day
    df_sales['DayOfWeek'] = df_sales['OrderDate'].dt.day_name()
    df_sales['Quarter'] = df_sales['OrderDate'].dt.to_period('Q').astype(str)
    print("Date features extracted.")

# Calculate TotalSale if not present (it was in the sample CSV but good practice to check)
if 'Quantity' in df_sales.columns and 'UnitPrice' in df_sales.columns and 'TotalSale' not in df_sales.columns:
    df_sales['TotalSale'] = df_sales['Quantity'] * df_sales['UnitPrice']
    print("'TotalSale' column calculated.")

display(df_sales.head())

## 5. Exploratory Data Analysis (EDA)


### 5.1 Univariate Analysis

#### Distribution of Numerical Features

In [ ]:
numerical_cols = df_sales.select_dtypes(include=np.number).columns.tolist()
print(f"Numerical columns: {numerical_cols}")

for col in ['Quantity', 'UnitPrice', 'TotalSale']:
    if col in df_sales.columns:
        fig = px.histogram(df_sales, x=col, title=f'Distribution of {col}', marginal='box')
        fig.show()


#### Counts of Categorical Features

In [ ]:
categorical_cols = df_sales.select_dtypes(include='object').columns.tolist()
# Also include 'MonthName', 'DayOfWeek', 'Quarter' if they were created
if 'MonthName' in df_sales.columns: categorical_cols.append('MonthName') # Example
print(f"Categorical columns for count plots: {categorical_cols}")

for col in ['ProductCategory', 'Salesperson', 'Region', 'MonthName']:
    if col in df_sales.columns:
        fig = px.bar(df_sales[col].value_counts().reset_index(), 
                     x='index', y=col, title=f'Counts of {col}',
                     labels={'index': col, col: 'Count'})
        fig.update_layout(xaxis_title=col, yaxis_title='Count')
        fig.show()

### 5.2 Bivariate and Multivariate Analysis

#### Sales Over Time

In [ ]:
if 'OrderDate' in df_sales.columns and 'TotalSale' in df_sales.columns:
    # Ensure OrderDate is sorted for time series plotting
    df_time_sales = df_sales.sort_values('OrderDate').set_index('OrderDate')['TotalSale'].resample('M').sum().reset_index()
    fig = px.line(df_time_sales, x='OrderDate', y='TotalSale', title='Monthly Sales Over Time', markers=True)
    fig.show()
    
    # Quarterly Sales
    df_quarterly_sales = df_sales.groupby('Quarter')['TotalSale'].sum().reset_index()
    fig_quarterly = px.bar(df_quarterly_sales, x='Quarter', y='TotalSale', title='Total Sales per Quarter')
    fig_quarterly.show()

#### Sales by Product Category

In [ ]:
if 'ProductCategory' in df_sales.columns and 'TotalSale' in df_sales.columns:
    category_sales = df_sales.groupby('ProductCategory')['TotalSale'].sum().sort_values(ascending=False).reset_index()
    fig = px.bar(category_sales, x='ProductCategory', y='TotalSale', 
                 title='Total Sales by Product Category', color='ProductCategory')
    fig.show()

#### Sales by Region

In [ ]:
if 'Region' in df_sales.columns and 'TotalSale' in df_sales.columns:
    region_sales = df_sales.groupby('Region')['TotalSale'].sum().sort_values(ascending=False).reset_index()
    fig = px.pie(region_sales, values='TotalSale', names='Region', title='Sales Distribution by Region', hole=0.3)
    fig.show()

#### Sales by Salesperson

In [ ]:
if 'Salesperson' in df_sales.columns and 'TotalSale' in df_sales.columns:
    salesperson_performance = df_sales.groupby('Salesperson')['TotalSale'].sum().sort_values(ascending=False).reset_index()
    fig = px.bar(salesperson_performance, x='Salesperson', y='TotalSale', 
                 title='Total Sales by Salesperson', color='Salesperson')
    fig.show()

#### Correlation Analysis (Numerical Features)

In [ ]:
if len(numerical_cols) > 1:
    # Select only numerical columns for correlation that make sense (e.g., exclude Year, Month if not desired for this specific heatmap)
    cols_for_corr = ['Quantity', 'UnitPrice', 'TotalSale']
    # Filter out columns that might not exist in the dummy data or are not suitable
    cols_for_corr = [col for col in cols_for_corr if col in df_sales.columns and pd.api.types.is_numeric_dtype(df_sales[col])]
    
    if len(cols_for_corr) > 1:
        correlation_matrix = df_sales[cols_for_corr].corr()
        fig = px.imshow(correlation_matrix, text_auto=True, aspect="auto",
                        title='Correlation Matrix of Numerical Features',
                        color_continuous_scale='RdBu_r') # Red-Blue diverging scale
        fig.show()
    else:
        print("Not enough numerical columns for a meaningful correlation matrix.")
else:
    print("No numerical columns found for correlation analysis.")

#### Product Performance (e.g., Top N Products by Sales)

In [ ]:
if 'ProductName' in df_sales.columns and 'TotalSale' in df_sales.columns:
    top_n = 10
    product_sales = df_sales.groupby('ProductName')['TotalSale'].sum().sort_values(ascending=False).head(top_n).reset_index()
    fig = px.bar(product_sales, x='ProductName', y='TotalSale', 
                 title=f'Top {top_n} Products by Sales', color='ProductName')
    fig.update_layout(xaxis_title='Product Name', yaxis_title='Total Sales')
    fig.show()

## 6. Summary of Initial Findings and Next Steps

Based on the exploration:

1.  **Data Quality:** (Comment on missing values, duplicates, data types after cleaning)
2.  **Key Trends:** (e.g., "Sales peak in Q4", "Electronics is the highest-grossing category")
3.  **Top Performers:** (e.g., "Salesperson X has the highest sales", "Product Y is the best-seller")
4.  **Regional Performance:** (e.g., "North region contributes the most to sales")
5.  **Potential Issues/Further Questions:** (e.g., "Why did sales dip in June?", "Is there a correlation between unit price and quantity sold for specific categories?")

**Next Steps:**
* Deeper dive into specific segments.
* Time series forecasting if applicable.
* Customer segmentation.
* Prepare data for BI dashboard (aggregations, specific metrics).